# SQL

Для анализа дан небольшой фаил размеров в 240 Мб содержащий только информацию о новостройках. Содержит срезы на каждый день(кроме дней поломок - простоя сервиса) Т.е. есть по каждому обекту есть информация за каждый день. Подучатеся в год(365 дней) информация об обект дублируется 365 раз.
Учтите что сложный селект может работать не быстро.

Для анализа использовать фаил с базой /home/synchronize_data/price_analytics/24.db.copied

In [1]:
!ls -la /home/synchronize_data/price_analytics/24.db.copied

-rw-r--r-- 1 synchronize_data synchronize_data 272957440 Jun 19 16:10 /home/synchronize_data/price_analytics/24.db.copied


In [14]:
import sqlite3

# Подключение к базе данных
connection = sqlite3.connect('/home/synchronize_data/price_analytics/24.db.copied')
cursor = connection.cursor()

# Запрос для получения списка всех таблиц
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Вывод имен таблиц
for table in tables:
    print(table[0])

# Закрытие подключения
connection.close()

copied


# Пример использования: задача вывести самую дорогую квартиру из имеющихся.

Запустите следующие 6 строчек, они должны отработать корректно.

In [8]:
!pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.1
[notice] To update, run: pip install --upgrade pip


In [17]:
import matplotlib.pyplot as plt
import sqlite3
import json

In [18]:
connection = sqlite3.connect('/home/synchronize_data/price_analytics/24.db.copied')
cursor = connection.cursor()

Поля рекомендованные:
            id,
            "title",
            "price",
            "price_sale",
            date(datetime_create) as date,
            "brand",
            "apartment_area",
            "apartment_floor",
            "apartment_floors_total",
            "apartment_room",
            "apartment_ppm"

In [33]:
cursor.execute("""
SELECT 
    *
FROM copied
where 
    category = "Новостройки" 
    or category = "Реновация"
ORDER BY price desc
LIMIT 1
""")

In [34]:
flat = cursor.fetchall()

In [35]:
flat

[(44871689,
  'Саввинская 27 | 27 | 2 | 340.9 | Саввинская 27 | 55.730824,37.557959 | 1',
  847419076,
  847419076,
  '2024-06-02 06:13:37.130681',
  'Новостройки',
  'level.ru',
  'https://level.ru/old/sav27/apartment/6room/5-1/',
  340.9,
  2,
  6,
  6,
  2485829)]

# Задачи:

Выведите список новостроек в порядке убывания количества доступных к покупке квартир.

In [62]:
cursor.execute("PRAGMA table_info(copied);")
columns = cursor.fetchall()
columns

[(0, 'id', 'INT', 0, None, 0),
 (1, 'title', 'TEXT', 0, None, 0),
 (2, 'price', 'NUM', 0, None, 0),
 (3, 'price_sale', 'NUM', 0, None, 0),
 (4, 'datetime_create', 'NUM', 0, None, 0),
 (5, 'category', 'TEXT', 0, None, 0),
 (6, 'brand', 'TEXT', 0, None, 0),
 (7, 'url', 'TEXT', 0, None, 0),
 (8, 'apartment_area', 'NUM', 0, None, 0),
 (9, 'apartment_floor', 'INT', 0, None, 0),
 (10, 'apartment_floors_total', 'INT', 0, None, 0),
 (11, 'apartment_room', 'INT', 0, None, 0),
 (12, 'apartment_ppm', 'INT', 0, None, 0)]

In [63]:
cursor.execute("""
SELECT 
    *
FROM copied
where 
    category = "Новостройки" 
ORDER BY title desc
LIMIT 3
""")

flat = cursor.fetchall()
flat

[(12658993,
  '№7, Корпус «Река», 2 этаж',
  58705556,
  58705556,
  '2023-05-13 21:18:37.281954',
  'Новостройки',
  'Галс-Девелопмент',
  'https://hals-development.ru/params/kosmodamianskaya/2533834',
  48.23,
  2,
  9,
  1,
  1217200),
 (12658994,
  '№7, Корпус «Река», 2 этаж',
  58705556,
  58705556,
  '2023-05-13 21:18:37.301288',
  'Новостройки',
  'Галс-Девелопмент',
  'https://hals-development.ru/params/kosmodamianskaya/2533834',
  48.23,
  2,
  9,
  1,
  1217200),
 (12658995,
  '№7, Корпус «Река», 2 этаж',
  58705556,
  58705556,
  '2023-05-13 21:18:37.312083',
  'Новостройки',
  'Галс-Девелопмент',
  'https://hals-development.ru/params/kosmodamianskaya/2533834',
  48.23,
  2,
  9,
  1,
  1217200)]

Выведите список новостроек в порядке убывания количества доступных к покупке квартир.

In [54]:
cursor.execute("""
SELECT 
    title,
    COUNT(*) AS count_apartments
FROM copied
WHERE 
    category = "Новостройки"
GROUP BY title
ORDER BY title DESC
LIMIT 5;
""")

flat = cursor.fetchall()
flat

[('№7, Корпус «Река», 2 этаж', 648),
 ('№34, Секция 3, 4 этаж', 224),
 ('Южнопортовая | 7 | 29 | 79.6 | г. Москва, Южнопортовая ул., вл. 28-28А | 55.705727,37.696445 | 280',
  22),
 ('Южнопортовая | 7 | 29 | 79.6 | Г. Москва, ул. Южнопортовая, вл.28-28а | 55.705727,37.696445 | 280',
  2),
 ('Южнопортовая | 7 | 29 | 33.3 | г. Москва, Южнопортовая ул., вл. 28-28А | 55.705727,37.696445 | 278',
  46)]

Найдите среднюю цену квадратного метра по всем новостройкам  

In [60]:
cursor.execute("""
SELECT 
    AVG(price) AS average_price
FROM copied
WHERE 
    category = "Новостройки";
""")

flat = cursor.fetchall()
flat

[(27342250.76069243,)]

Выведите список новостроек в порядке убывания общего количества квартир.  

Найдите среднюю площадь квартир в новостройках с количеством комнат больше 3  

Найдите среднюю цену квадратного метра по всем новостройкам на конкретный день.  

Найдите среднюю цен за квадратный метр в новостройках с количеством этажей больше 15.  

Найти все новостройки которые появились в 2023 году  

Найти все новостройки которые имеют больше 50 квартир  

Найдите максимальную и минимальную цену среди всех новостроек  

Найти все новостройки в которых есть квартиры площадью больше 100 кв метров  

Найти новостройки котрые были построекны застройщщиком Донстрой  

Найти всех застройщиков которые построили более 5 новостроек в 2023 году  

Вывести список жилых комплексов с наибольшим количеством проданных квартир за последние 3 месяца. Вывести количество проданных квартир и процент от общего количества квартир этого комплекса

# Задачи с обогащением:

Построить месячный график количества продаж новостроек за последний год.  

Выведите список новостроек в западном районе.  

Найти все новостройки в ЗАО и СЗАО которые имеют больше 10 этажей  

Найти самый популярный размер квартир в каждом округе Москвы, и посчитать количеств продаж каждого размера ( сейчас за размер принимаем количество комнат)  

Найти все новостройки в пределах заданного радиуса от опеределнного места. И определить расстояние от каждой новостройки до заданного места, распределив их по возрастанию расстояния до заданного места.  

Построить график средней стоимости квадратного метра по месяцам, начиная с q2 2023 по q2 2024  

Найти все новостройки в которых есть квартиры с видом на определенный объект ( например парк, реку, останкинскую телебашню, москва сити или другую достопримечательность).

Показать информацию о количестве квартир с таким видом их средние площади и стоимости.  

Вывести список жилых комплексов с наибольшей разницей между стоимостью квартир в них и среденей стоимостью квартир в районе. Рассчитать процентное отклонение цены каждой квартиры от средней цены по району.  

Начиная с 2023 года, построить несколько графиков по месяцам : стоимости кв метра квартир, курса доллара(сколько рублей за один доллар), ключевой ставки, уровень безработиц, численность населения в городе, среднее количество детей в семье, количество сделок по ипотеке, количество сделок по льготным ипотечным программам,  

Найти все новостройки информация о которых появилась в 2023 году имеют рейтинг выше среднего по городу. Вывести список этих новостроек, среднее, минимальное и максимальное количество этажей в них, в разрезе типа квартир ( тип это количество комнат ) с количеством квартир.  

Вывести весь жк квартиры в которых были введены в эксплуатацию с чистовой отделкой в 2024 году  

вывести список жк бизнес класса в ВАО города Москвы района Измайлово, ключи от которых владельцы получили в 2023  

Выведите жк где соотношение количества квартир к количеств парковочных мест меньше 2.5  

Введите количество квартир в кирпичных домах города Москвы  

Определите среднее время оживания лита в домах застройщика level  

Выведите список жк от которых до ближайшей станции метро 10 минут пешком  

Выведите общее число объектов застройщиков которые больше 20 лет на рынке, и при качестве объектов выше среднего.

# Задачи на подумать и предложить

Выдвинуть 10 гипотез и проверить их на основе данных ( если данных не хватает то добавить данными из интернета)  

Гипотеза 1 - стоимость квартиры зависит от площади  

Гипотеза 2 - стоимость квартиры зависит от расстояния до центра города  

Гипотеза 3 -  

In [9]:
class Tree:

    def __init__(self, height, leaf_colour, age, form):
        self.height = height
        self.leaf_colour = leaf_colour
        self.age = age
        self.form = form


    # def make_older(self):
    #     self

tree1 = Tree(20, "red", 100, "triangle")

tree1.leaf_colour

'red'